# Dense-X-Retrieval Pack

This notebook walks through using the `DenseXRetrievalPack`, which parses documents into nodes, and then generates propositions from each node to assist with retreival.

This follows the idea from the paper [Dense X Retrieval: What Retreival Granularity Should We Use?](https://arxiv.org/abs/2312.06648).

From the paper, a proposition is described as:

```
Propositions are defined as atomic expressions within text, each encapsulating a distinct factoid and presented in a concise, self-contained natural language format.
```

We use the provided OpenAI prompt from their paper to generate propositions, which are then embedded and used to retrieve their parent node chunks.

## Setup

In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-readers-file

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

import nest_asyncio

nest_asyncio.apply()

In [ ]:
!mkdir -p 'data/'
!curl 'https://arxiv.org/pdf/2307.09288.pdf' -o 'data/llama2.pdf'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.0M  100 13.0M    0     0  1160k      0  0:00:11  0:00:11 --:--:-- 1574k


In [ ]:
from llama_index.readers.file import UnstructuredReader

documents = UnstructuredReader().load_data("data/llama2.pdf")

[nltk_data] Downloading package punkt to /home/loganm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/loganm/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/home/loganm/.cache/pypoetry/virtualenvs/llama-hub-86aDfznI-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Run the DenseXRetrievalPack

The `DenseXRetrievalPack` creates both a retriver and query engine.

In [ ]:
from llama_index.core.llama_pack import download_llama_pack

DenseXRetrievalPack = download_llama_pack("DenseXRetrievalPack", "./dense_pack")

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

dense_pack = DenseXRetrievalPack(
    documents,
    proposition_llm=OpenAI(model="gpt-3.5-turbo", max_tokens=750),
    query_llm=OpenAI(model="gpt-3.5-turbo", max_tokens=256),
    text_splitter=SentenceSplitter(chunk_size=1024),
)
dense_query_engine = dense_pack.query_engine

Generating embeddings: 100%|██████████| 2210/2210 [00:13<00:00, 159.12it/s]


In [ ]:
from llama_index.core import VectorStoreIndex

base_index = VectorStoreIndex.from_documents(documents)
base_query_engine = base_index.as_query_engine()

## Test Queries

### How was Llama2 pretrained?

In [ ]:
from IPython.display import Markdown, display

response = dense_query_engine.query("How was Llama2 pretrained?")
display(Markdown(str(response)))

Llama 2 was pretrained using an optimized auto-regressive transformer. Several changes were made to improve performance, including more robust data cleaning, updated data mixes, training on 40% more total tokens, doubling the context length, and using grouped-query attention (GQA) to improve inference scalability for larger models.

In [ ]:
response = base_query_engine.query("How was Llama2 pretrained?")
display(Markdown(str(response)))

Llama 2 was pretrained using an optimized auto-regressive transformer. The pretraining approach involved robust data cleaning, updated data mixes, training on 40% more total tokens, doubling the context length, and using grouped-query attention (GQA) to improve inference scalability for larger models. The training corpus included a new mix of data from publicly available sources, excluding data from Meta's products or services. The pretraining methodology and training details are described in more detail in the provided context.

### What baselines are used to compare performance and accuracy?

In [ ]:
response = dense_query_engine.query(
    "What baselines are used to compare performance and accuracy?"
)
display(Markdown(str(response)))

The baselines used to compare performance and accuracy are Llama 1, Falcon, and MPT. These models are compared with Llama 2 to evaluate its advancements in terms of safety and other important aspects.

In [ ]:
response = base_query_engine.query(
    "What baselines are used to compare performance and accuracy?"
)
display(Markdown(str(response)))

The baselines used to compare performance and accuracy are the MPT and Falcon models. These models are compared against the evaluation framework and any publicly reported results to determine the best score.

### What datasets were used for measuring performance and accuracy?

In [ ]:
response = dense_query_engine.query(
    "What datasets were used for measuring performance and accuracy?"
)
display(Markdown(str(response)))

The datasets used for measuring performance and accuracy include HumanEval, MBPP, PIQA, SIQA, HellaSwag, WinoGrande, ARC easy and challenge, OpenBookQA, CommonsenseQA, NaturalQuestions, TriviaQA, SQuAD, GSM8K, and MATH.

In [ ]:
response = base_query_engine.query(
    "What datasets were used for measuring performance and accuracy?"
)
display(Markdown(str(response)))

The datasets used for measuring performance and accuracy include HumanEval, MBPP, PIQA, SIQA, HellaSwag, WinoGrande, ARC easy and challenge, OpenBookQA, CommonsenseQA, NaturalQuestions, TriviaQA, SQuAD, QuAC, BoolQ, GSM8K, and MATH.